In [ ]:
from __init__ import *

In [ ]:
directory_path = "{}/feature/".format(SETTING["directory_path"])

kraft.path.path(directory_path)

In [ ]:
for url in ("https://ndownloader.figshare.com/files/22629137",):

    print(kraft.internet.download(url, directory_path))

## Map cell line name

In [ ]:
name_to_id = (
    pd.read_csv(
        "data/cell-line-synon-names.txt",
        sep="\t",
        usecols=(0, 1),
        index_col=1,
        squeeze=True,
    )
    .dropna()
    .to_dict()
)

id_to_rename = (
    pd.read_csv(
        "data/cell-line-best-names.txt",
        sep="\t",
        usecols=(0, 1),
        index_col=0,
        squeeze=True,
    )
    .dropna()
    .to_dict()
)

name_to_rename = {name.lower(): id_to_rename[id_] for name, id_ in name_to_id.items()}

len(name_to_rename)

In [ ]:
for name, rename in (
    pd.read_csv(
        "data/ice000.cols.cell_line_identity.2777.csv",
        usecols=(2, 3),
        index_col=0,
        squeeze=True,
    )
    .dropna()
    .items()
):

    name_lower = name.lower()

    if name_lower in name_to_rename:

        rename_now = name_to_rename[name_lower]

        if rename_now != rename:

            print("{}: {} ==> {}".format(name, rename_now, rename))

    name_to_rename[name_lower] = rename

len(name_to_rename)

In [ ]:
feature_x_cell_line = pd.read_csv(
    "{}/sample_info.csv".format(directory_path), index_col=1
).T

In [ ]:
for rename, names in feature_x_cell_line.iloc[:3, :].items():

    for name in names.dropna():

        name_lower = name.lower()

        if name_lower in name_to_rename:

            rename_now = name_to_rename[name_lower]

            if rename_now != rename:

                print("{}: {} ==> {}".format(name, rename_now, rename))

        name_to_rename[name_lower] = rename

len(name_to_rename)

In [ ]:
for rename in tuple(name_to_rename.values()):

    name_to_rename[rename.lower()] = rename

len(name_to_rename)

In [ ]:
name_to_rename = pd.Series(name_to_rename).sort_values()

name_to_rename.index.name = "Name"

name_to_rename.name = "Rename"

name_to_rename.to_csv(
    "{}/cell_line_name_rename.tsv".format(directory_path), sep="\t", header=True
)

name_to_rename

## Make feature_x_cell_line

### All

In [ ]:
feature_x_cell_line.loc["Is Fibroblast", :] = tuple(
    int(bool(re.match(".*FIBROBLAST$", name)))
    for name in feature_x_cell_line.loc["CCLE_Name", :].to_numpy()
)

feature_x_cell_line.index.name = "Feature"

feature_x_cell_line.columns.name = "Cell Line"

kraft.pandas.error_axes(feature_x_cell_line)

feature_x_cell_line.to_csv(
    "{}/feature_x_cell_line.tsv".format(directory_path), sep="\t"
)

feature_x_cell_line

### Continuous

In [ ]:
feature_x_cell_line_x_continuous = pd.concat((feature_x_cell_line.loc[["age"], :],))

feature_x_cell_line_x_continuous.to_csv(
    "{}/feature_x_cell_line_x_continuous.tsv".format(directory_path), sep="\t"
)

feature_x_cell_line_x_continuous

### Binary

In [ ]:
dataframes = [feature_x_cell_line.loc[["Is Fibroblast"], :]]

for label in (
    "sex",
    "sample_collection_site",
    "primary_or_metastasis",
    "primary_disease",
    "Subtype",
    "lineage",
    "lineage_subtype",
    "lineage_sub_subtype",
    "lineage_molecular_subtype",
):

    dataframe = kraft.pandas.binarize(feature_x_cell_line.loc[label, :])

    dataframe.index = (
        "({}) {}".format(label, label_) for label_ in dataframe.index.to_numpy()
    )

    dataframes.append(dataframe)

feature_x_cell_line_x_01 = pd.concat(dataframes)

feature_x_cell_line_x_01.index.name = "Feature"

feature_x_cell_line_x_01.to_csv(
    "{}/feature_x_cell_line_x_01.tsv".format(directory_path), sep="\t"
)

feature_x_cell_line_x_01